In [1]:
#kerasをインポートする
from keras import backend
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense,Activation,LSTM
from keras.utils import np_utils
from keras.models import load_model    
from keras.layers.core import Dropout

#pandasとnumpyをインポートする
import pandas as pd
import numpy as np

#すべての警告を非表示にする
import warnings
warnings.filterwarnings('ignore')

#Word2Vecをインポートする
from gensim.models.word2vec import Word2Vec

#MeCab:形態素解析器のインポート
import MeCab

In [2]:
class LstmLearnClass:
    def __init__(self):
        #中間総数
        self.n_hidden = 32
        #入力データ数
        self.data_dim = 50
        self.timesteps = 30
        #学習回数
        self.epochs = 1000   
        #出力データ数
        self.num_classes = 6
        
    def dataInput(self):
        #csv1ファイルを読み取る
        survey = pd.read_csv("survey.csv")
        #欠損値を削除
        survey = survey.dropna()
        survey.isna().sum()
        
        return survey
    
    def dataXtrain(self,survey):
        #Word2Vecで使用するデータのパス指定
        model_path = './word2vec.gensim.model'
        w2v_model = Word2Vec.load(model_path)
        #単語をベクトルに変換
        x_train =[]
        for survey_row in survey:
            x_train.append(w2v_model[survey_row])
        x_train = np.array(x_train)
        
        return x_train
        
    def dataYtrain(self,survey):
        #教師データを設定
        y_train = survey['satisfaction']
        y_train = np_utils.to_categorical(y_train)
        
        return y_train
        
    def lstmModel(self):
        #LSTMの設定
        lstm_model = Sequential()
        lstm_model.add(LSTM(self.n_hidden, return_sequences=True,
        input_shape=(self.timesteps, self.data_dim)))  # 32次元のベクトルのsequenceを出力する
        lstm_model.add(LSTM(self.n_hidden))  # 32次元のベクトルを一つ出力する
        lstm_model.add(Dense(self.num_classes, activation='softmax'))

        lstm_model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        
        return lstm_model

    def morphologicAlanalysis(self,survey):
        #形態素解析の実行
        words_arr2 = []
        for suvey_row in range(len(survey)):
            text = survey["comment"].iloc[suvey_row]
            tagger = MeCab.Tagger()
            words = tagger.parse(text).splitlines()

            words_arr = []
            for i in range(self.timesteps):
                if(i < len(words)):
                    word_tmp = words[i].split()[0]
                    words_arr.append(word_tmp)
                else:
                    #入力データの余った配列に文字列を設定する
                    word_tmp = 'A'
                    words_arr.append(word_tmp)
            words_arr2.append(words_arr)
        return words_arr2

    def lstmLearn(self,lstm_model,x_train,y_train):
        #LSTMの重さ更新
        lstm_model.fit(x_train, y_train,batch_size=64, epochs=self.epochs)
        #LSTMの重さ保存
        lstm_model.save_weights('w2v_lstm_weights.h5')

In [3]:
def main():
    #LSTM実行
    lstm = LstmLearnClass()
    survey = lstm.dataInput()
    x_train = lstm.morphologicAlanalysis(survey)
    X = lstm.dataXtrain(x_train)
    Y = lstm.dataYtrain(survey)
    #LSTMの重さ保存
    lstm.lstmLearn(lstm.lstmModel(),X,Y)

if __name__ == '__main__':
    main()

Epoch 1/1000
2/2 [==============================] - 3s 28ms/step - loss: 1.7781 - accuracy: 0.1451
Epoch 2/1000
2/2 [==============================] - 0s 28ms/step - loss: 1.7502 - accuracy: 0.1606
Epoch 3/1000
2/2 [==============================] - 0s 25ms/step - loss: 1.7300 - accuracy: 0.3187
Epoch 4/1000
2/2 [==============================] - 0s 25ms/step - loss: 1.7071 - accuracy: 0.3213
Epoch 5/1000
2/2 [==============================] - 0s 30ms/step - loss: 1.6859 - accuracy: 0.3109
Epoch 6/1000
2/2 [==============================] - 0s 28ms/step - loss: 1.6691 - accuracy: 0.3161
Epoch 7/1000
2/2 [==============================] - 0s 30ms/step - loss: 1.6644 - accuracy: 0.2901
Epoch 8/1000
2/2 [==============================] - 0s 31ms/step - loss: 1.6518 - accuracy: 0.2746
Epoch 9/1000
2/2 [==============================] - 0s 33ms/step - loss: 1.6417 - accuracy: 0.2978
Epoch 10/1000
2/2 [==============================] - 0s 27ms/step - loss: 1.6255 - accuracy: 0.3057
Epoch 11/

2/2 [==============================] - 0s 38ms/step - loss: 1.2267 - accuracy: 0.4844
Epoch 84/1000
2/2 [==============================] - 0s 25ms/step - loss: 1.1938 - accuracy: 0.4922
Epoch 85/1000
2/2 [==============================] - 0s 26ms/step - loss: 1.1846 - accuracy: 0.5103
Epoch 86/1000
2/2 [==============================] - 0s 25ms/step - loss: 1.1923 - accuracy: 0.5128
Epoch 87/1000
2/2 [==============================] - 0s 28ms/step - loss: 1.2038 - accuracy: 0.4897
Epoch 88/1000
2/2 [==============================] - 0s 23ms/step - loss: 1.1592 - accuracy: 0.5052
Epoch 89/1000
2/2 [==============================] - 0s 26ms/step - loss: 1.2113 - accuracy: 0.4975
Epoch 90/1000
2/2 [==============================] - 0s 29ms/step - loss: 1.1767 - accuracy: 0.5207
Epoch 91/1000
2/2 [==============================] - 0s 26ms/step - loss: 1.2576 - accuracy: 0.5025
Epoch 92/1000
2/2 [==============================] - 0s 26ms/step - loss: 1.1434 - accuracy: 0.5259
Epoch 93/1000


2/2 [==============================] - 0s 28ms/step - loss: 1.1711 - accuracy: 0.6165
Epoch 165/1000
2/2 [==============================] - 0s 31ms/step - loss: 1.0036 - accuracy: 0.5724
Epoch 166/1000
2/2 [==============================] - 0s 25ms/step - loss: 1.0203 - accuracy: 0.5440
Epoch 167/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.9830 - accuracy: 0.6426
Epoch 168/1000
2/2 [==============================] - 0s 27ms/step - loss: 1.3180 - accuracy: 0.4948
Epoch 169/1000
2/2 [==============================] - 0s 29ms/step - loss: 1.2508 - accuracy: 0.4922
Epoch 170/1000
2/2 [==============================] - 0s 31ms/step - loss: 1.0278 - accuracy: 0.6477
Epoch 171/1000
2/2 [==============================] - 0s 26ms/step - loss: 1.0360 - accuracy: 0.5700
Epoch 172/1000
2/2 [==============================] - 0s 31ms/step - loss: 1.1726 - accuracy: 0.5076
Epoch 173/1000
2/2 [==============================] - 0s 29ms/step - loss: 1.0578 - accuracy: 0.5440
Epoch

2/2 [==============================] - 0s 29ms/step - loss: 0.5756 - accuracy: 0.7799
Epoch 246/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.5625 - accuracy: 0.7746
Epoch 247/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.5467 - accuracy: 0.7928
Epoch 248/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.5237 - accuracy: 0.7980
Epoch 249/1000
2/2 [==============================] - 0s 26ms/step - loss: 0.5631 - accuracy: 0.7877
Epoch 250/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.4902 - accuracy: 0.8239
Epoch 251/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.4958 - accuracy: 0.7928
Epoch 252/1000
2/2 [==============================] - 0s 26ms/step - loss: 0.5347 - accuracy: 0.7850
Epoch 253/1000
2/2 [==============================] - 0s 25ms/step - loss: 0.4602 - accuracy: 0.8187
Epoch 254/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.5114 - accuracy: 0.8083
Epoch

2/2 [==============================] - 0s 29ms/step - loss: 0.3252 - accuracy: 0.8808
Epoch 327/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.4087 - accuracy: 0.8394
Epoch 328/1000
2/2 [==============================] - 0s 23ms/step - loss: 0.3049 - accuracy: 0.8808
Epoch 329/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.4153 - accuracy: 0.8367
Epoch 330/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.3764 - accuracy: 0.8653
Epoch 331/1000
2/2 [==============================] - 0s 25ms/step - loss: 0.3053 - accuracy: 0.8860
Epoch 332/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.3748 - accuracy: 0.8549
Epoch 333/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.3304 - accuracy: 0.8756
Epoch 334/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.2895 - accuracy: 0.9067
Epoch 335/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.3992 - accuracy: 0.8212
Epoch

...

2/2 [==============================] - 0s 30ms/step - loss: 0.0606 - accuracy: 0.9870
Epoch 651/1000
2/2 [==============================] - 0s 35ms/step - loss: 0.0949 - accuracy: 0.9767
Epoch 652/1000
2/2 [==============================] - 0s 25ms/step - loss: 0.0881 - accuracy: 0.9741
Epoch 653/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.0654 - accuracy: 0.9870
Epoch 654/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.2301 - accuracy: 0.9534
Epoch 655/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.0735 - accuracy: 0.9793
Epoch 656/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.0774 - accuracy: 0.9870
Epoch 657/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.1462 - accuracy: 0.9197
Epoch 658/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.1015 - accuracy: 0.9663
Epoch 659/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.2613 - accuracy: 0.9327
Epoch

2/2 [==============================] - 0s 39ms/step - loss: 0.0269 - accuracy: 0.9870
Epoch 732/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0313 - accuracy: 0.9870
Epoch 733/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0298 - accuracy: 0.9870
Epoch 734/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.0292 - accuracy: 0.9870
Epoch 735/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.0306 - accuracy: 0.9870
Epoch 736/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.0308 - accuracy: 0.9870
Epoch 737/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.0309 - accuracy: 0.9870
Epoch 738/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.0307 - accuracy: 0.9870
Epoch 739/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0300 - accuracy: 0.9870
Epoch 740/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.0289 - accuracy: 0.9870
Epoch

2/2 [==============================] - 0s 35ms/step - loss: 0.0244 - accuracy: 0.9870
Epoch 813/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0206 - accuracy: 0.9922
Epoch 814/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.0242 - accuracy: 0.9922
Epoch 815/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.0272 - accuracy: 0.9870
Epoch 816/1000
2/2 [==============================] - 0s 39ms/step - loss: 0.0230 - accuracy: 0.9922
Epoch 817/1000
2/2 [==============================] - 0s 37ms/step - loss: 0.0279 - accuracy: 0.9870
Epoch 818/1000
2/2 [==============================] - 0s 35ms/step - loss: 0.0204 - accuracy: 0.9922
Epoch 819/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0278 - accuracy: 0.9870
Epoch 820/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0245 - accuracy: 0.9870
Epoch 821/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.0276 - accuracy: 0.9870
Epoch

2/2 [==============================] - 0s 33ms/step - loss: 0.0218 - accuracy: 0.9922
Epoch 894/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0260 - accuracy: 0.9870
Epoch 895/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.0213 - accuracy: 0.9922
Epoch 896/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0258 - accuracy: 0.9870
Epoch 897/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.0218 - accuracy: 0.9922
Epoch 898/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.0259 - accuracy: 0.9870
Epoch 899/1000
2/2 [==============================] - 0s 37ms/step - loss: 0.0217 - accuracy: 0.9922
Epoch 900/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.0220 - accuracy: 0.9870
Epoch 901/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.0256 - accuracy: 0.9870
Epoch 902/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.0253 - accuracy: 0.9870
Epoch

2/2 [==============================] - 0s 29ms/step - loss: 0.0243 - accuracy: 0.9870
Epoch 975/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.0242 - accuracy: 0.9870
Epoch 976/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0211 - accuracy: 0.9922
Epoch 977/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.0244 - accuracy: 0.9870
Epoch 978/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.0205 - accuracy: 0.9922
Epoch 979/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.0245 - accuracy: 0.9870
Epoch 980/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.0240 - accuracy: 0.9870
Epoch 981/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.0247 - accuracy: 0.9870
Epoch 982/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0204 - accuracy: 0.9870
Epoch 983/1000
2/2 [==============================] - 0s 36ms/step - loss: 0.0208 - accuracy: 0.9922
Epoch